In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql 
SELECT name,
       type
FROM sqlite_master
WHERE type IN ("table", "view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql 
SELECT * FROM track
LIMIT 5;

Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


# Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

<table style="width:20%">
<tr>
<th>Artist Name</th>	<th>Genre</th>
</tr>

<tr>
<td>Regal</td>	
<td>Hip-Hop</td>
</tr>
<tr>
<td>Red Tone</td>	<td>Punk</td>
</tr>
<tr>
<td>Meteor and the Girls</td>	<td>Pop</td>
</tr>
<tr>
<td>Slim Jim Bites</td>	<td>Blues</td>
</table>

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [4]:
%%sql
WITH customer_in_usa AS (
    SELECT il.*
    FROM customer c INNER JOIN invoice i ON c.customer_id = i.customer_id
    INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
    WHERE c.country="USA"
    )
   
       
SELECT g.name genre_name,
       COUNT(ciu.invoice_line_id) tracks_sold_usa,
       ROUND((CAST(COUNT(ciu.invoice_line_id) as FLOAT) /(SELECT COUNT(*) from customer_in_usa)*100),2) percentages
FROM customer_in_usa ciu INNER JOIN track t ON ciu.track_id = t.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1
ORDER BY 2 DESC;

Done.


genre_name,tracks_sold_usa,percentages
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


As per table above, Rock represents for more than half of the tracks sold in the USA - so we should focus on artists in this genre

In [5]:
%%sql

SELECT ar.name artist_name,
       COUNT(al.album_id) albums_sold
FROM artist ar
INNER JOIN album al ON ar.artist_id = al.artist_id
INNER JOIN track t ON t.album_id = al.album_id
INNER JOIN genre g ON t.genre_id = g.genre_id
WHERE g.name = "Rock"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 3;

Done.


artist_name,albums_sold
Led Zeppelin,114
U2,112
Deep Purple,92


As per analysis, the top 3 best sellers Rock bands are:
- Led Zeppelin
- U2
- Deep Purple 

We should purchase albums from the 3 above bands to insure our return

# Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [6]:
%%sql
WITH total_sales_employee AS (
    SELECT ROUND(SUM(i.total),2) total_sales,
           c.support_rep_id 
    FROM invoice i INNER JOIN customer c ON c.customer_id = i.customer_id
    GROUP BY 2
    )

SELECT  e.first_name || " " || e.last_name  employee_name, 
        e.city,
        e.hire_date,
        tse.total_sales
FROM employee e INNER JOIN total_sales_employee tse ON e.employee_id = tse.support_rep_id
WHERE e.title = 'Sales Support Agent';

Done.


employee_name,city,hire_date,total_sales
Jane Peacock,Calgary,2017-04-01 00:00:00,1731.51
Margaret Park,Calgary,2017-05-03 00:00:00,1584.0
Steve Johnson,Calgary,2017-10-17 00:00:00,1393.92


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

# Analyzing Sales by Country

Your next task is to analyze the sales data for customers from each different country. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis. You can use the following 'trick' to force the ordering of "Other" to last in your analysis.

In [7]:
%%sql
WITH country_or_other AS (
    SELECT 
        CASE
            WHEN 
            (SELECT count(*) from customer
            WHERE country = c.country)= 1 THEN "Other"
            ELSE c.country
        END AS country,
        c.customer_id,
        il.*
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
)

SELECT country,
       total_customers,
       avg_sales_per_customer,
       avg_order_value
FROM(
SELECT
    country,
    CASE
        WHEN country = "Other" THEN 1
        ELSE 0
    END as sort,
    count(distinct customer_id) total_customers,
    SUM(unit_price) total_sales,
    ROUND(SUM(unit_price) / count(distinct customer_id),2) avg_sales_per_customer,
    ROUND(SUM(unit_price) / count(distinct invoice_id),2) avg_order_value
FROM country_or_other
GROUP BY 1
ORDER BY 2, 3 DESC);

Done.


country,total_customers,avg_sales_per_customer,avg_order_value
USA,13,80.04,7.94
Canada,8,66.95,7.05
Brazil,5,85.54,7.01
France,5,77.81,7.78
Germany,4,83.66,8.16
United Kingdom,3,81.84,8.77
Czech Republic,2,136.62,9.11
India,2,91.58,8.72
Portugal,2,92.57,6.38
Other,15,73.0,7.45


Based on our analysis, our store has gained popularity in USA and surprisingly, people Czech Republic spends more on average. However, we shouldn't use the numbers in Czech since there are only 2 customers, it may be not accurate if we have more. We need more data for deeper analysis

# Album vs Individual Tracks

In [8]:
%%sql
WITH invoice_first_track AS (
    SELECT invoice_id,
           MIN(track_id) first_track
    FROM invoice_line
    GROUP BY 1
    )

SELECT
    album_purchase,
    count(invoice_id) total_invoices,
    cast(count(invoice_id) as FLOAT) / (SELECT COUNT(*) FROM invoice) percent
FROM
(SELECT 
    ift.*,
    CASE
        WHEN 
        (SELECT t.track_id
         FROM track t
         WHERE t.album_id = (SELECT t1.album_id FROM track t1
                            WHERE t1.track_id = ift.first_track)
         
         EXCEPT
         
         SELECT il1.track_id
         FROM invoice_line il1
         WHERE il1.invoice_id = ift.invoice_id
        ) IS NULL
        
        AND
        
        (SELECT il1.track_id FROM invoice_line il1
        WHERE il1.invoice_id = ift.invoice_id
        
        EXCEPT
        
        SELECT t.track_id FROM track t
        WHERE t.album_id = (SELECT t1.album_id FROM track t1
                           WHERE t1.track_id = ift.first_track) 
        ) IS NULL
        
            THEN "Yes"
            ELSE "No"
        END AS album_purchase
        FROM invoice_first_track ift
        )
GROUP BY 1;

Done.


album_purchase,total_invoices,percent
No,500,0.8143322475570033
Yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.